### Imports

In [8]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [9]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [10]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [11]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [12]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [13]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-83    8.3   100    0.72   0        Ruim     1.00      
-89    5.0   58     0.84   0        Ruim     1.00      
-69    5.7   33     0.57   0        Ruim     0.99      
-85    7.5   62     0.81   0        Ruim     1.00      
-88    4.0   36     0.81   0        Ruim     1.00      
-70    4.5   67     0.89   0        Ruim     1.00      
-84    9.3   42     0.73   0        Ruim     1.00      
-70    9.8   98     0.63   0        Ruim     1.00      
-65    9.8   42     0.43   0        Ruim     1.00      
-64    5.4   98     0.90   0        Ruim     1.00      
-74    7.2   40     0.91   0        Ruim     1.00      
-73    7.2   40     0.37   0        Ruim     0.99      
-71    9.0   74     0.58   0        Ruim     1.00      
-82    8.4   90     0.96   0        Ruim     1.00      
-61    6.2   49     0.44   0        Ruim     0.99      
-87    8.3   83     0.70   0        Ruim     1.0

### Save

In [14]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
